In [1]:
# export
from pathlib import Path
import re
import os
import json

In [2]:
# export
def read_nb(fname):
    "Read the notebook in `fname`."

    data = json.loads(Path(fname).read_text())

    for c in data["cells"]:
        c["source"] = "".join(c["source"])

    return data

In [3]:
# export

pattern: str = r"\s*\#\s*export[s]?"
pat = re.compile(pattern, re.IGNORECASE | re.MULTILINE)

In [4]:
# export
def nb2script(nb_path, script_path, replace=True, cli=False):

    nb_path = Path(nb_path)
    script_path = Path(script_path)

    if script_path.exists() and replace == False:
        raise FileExistsError(f"The file {script_path} already exists.")

    data = read_nb(nb_path)

    cells = [
        c["source"]
        for c in data["cells"]
        if c["cell_type"] == "code" and pat.search(c["source"]) is not None
    ]

    with open(script_path, "w") as f:
        if cli:

            f.write(f"#!/usr/bin/env python3" + "\n\n")

        f.write(
            f"# AUTOGENERATED! DO NOT EDIT! File to edit: {str(nb_path)} (unless otherwise specified)."
            + "\n"
        )
        for c in cells:
            f.write(pat.sub("", c) + "\n\n")

In [5]:
# export
def main(nb_path, script_path=None, replace=True, cli=False):

    print(f"Converting {str(nb_path)} to {str(script_path)}, replace={replace}, cli={cli}")

    nb_path = Path(nb_path)
    if not script_path:
        script_path = nb_path.with_suffix(".py")

    nb2script(nb_path, script_path, replace=replace, cli=cli)

In [6]:
main("./nb2script.ipynb", "./nb2s.py")

Converting ./nb2script.ipynb to ./nb2s.py, replace=True, cli=False


In [7]:
main("./nb2script.ipynb", "/Users/r/dotfiles/bin/nb2s", cli=True)

Converting ./nb2script.ipynb to /Users/r/dotfiles/bin/nb2s, replace=True, cli=True


In [ ]:
# export
if __name__ == "__main__":
    import argparse

    parser = argparse.ArgumentParser()

    parser.add_argument(
        "--replace",
        help="wether to replace the script or not",
        action="store_true",
        default=True,
    )

    parser.add_argument(
        "--cli",
        help="add shebang when the output will be used as a CLI",
        action="store_true",
        default=False,
    )

    parser.add_argument(
        "-n",
        "--nb",
        help="path to the notebook to convert",
        type=str,
        default=None,
        required=True,
    )

    parser.add_argument(
        "-s",
        "--script",
        help="path where the script will be saved",
        type=str,
        default=None,
        required=True,
    )

    args = parser.parse_args()

    exit(main(args.nb, args.script, args.replace, args.cli))